# PY

In [ ]:

"""
# -- --------------------------------------------------------------------------------------------------- -- #
# -- project: A SHORT DESCRIPTION OF THE PROJECT                                                         -- #
# -- script: data.py : python script for data collection                                                 -- #
# -- author: YOUR GITHUB USER NAME                                                                       -- #
# -- license: GPL-3.0 License                                                                            -- #
# -- repository: YOUR REPOSITORY URL                                                                     -- #
# -- --------------------------------------------------------------------------------------------------- -- #
"""

# Load packages
import pandas as pd
import numpy as np
from os import listdir, path
from os.path import isfile, join

# Get the absolute path of the folder
abspath = path.abspath("files/")

# Read all the files within the files folder
files = [f[8:-4] for f in listdir(abspath) if isfile(join(abspath, f))]

# Chronologically ordered files list
files = ["NAFTRAC_" + i.strftime("%Y%m%d") for i in sorted(pd.to_datetime(files))]

# Read and store all the files in a dictionary
data_files = {}
for i in files:
    # Read the file
    data = pd.read_csv("files/" + i + ".csv", skiprows=2, header=0)
    # Select only not null columns
    data = data.loc[:, pd.notnull(data.columns)]
    # Clean the ticker name for later use with yfinance
    data["Ticker"] = [i.replace("*","") for i in data["Ticker"]]
    # Weight as decimal
    data["Peso (%)"] = [i/100 for i in data["Peso (%)"]]
    # Save current file in dictionary of data
    data_files[i] = data



In [ ]:
"""
# -- --------------------------------------------------------------------------------------------------- -- #
# -- project: A SHORT DESCRIPTION OF THE PROJECT                                                         -- #
# -- script: functions.py : python script with general functions                                         -- #
# -- author: YOUR GITHUB USER NAME                                                                       -- #
# -- license: GPL-3.0 License                                                                            -- #
# -- repository: YOUR REPOSITORY URL                                                                     -- #
# -- --------------------------------------------------------------------------------------------------- -- #
"""

import pandas as pd
import numpy as np
import yfinance as yf
import time
import datetime

# Get Dates
def f_dates(p_files):
    # Labels for dataframe and yfinance
    t_dates = [i.strftime("%d-%m-%Y") for i in sorted([pd.to_datetime(i[8:]).date() for i in p_files])]

    # For other calculations
    i_dates = [i.strftime("%Y-%m-%d") for i in sorted([pd.to_datetime(i[8:]).date() for i in p_files])]

    # Final data to return
    r_f_dates = {"i_dates": i_dates, "t_dates": t_dates}

    return r_f_dates

#%%
# Get Tickers
def f_tickers(p_archivos, p_data_archivos):
    tickers = []
    for i in p_archivos:
        l_tickers = list(p_data_archivos[i]["Ticker"])
        [tickers.append(i + ".MX") for i in l_tickers]
    global_tickers = np.unique(tickers).tolist()

    # Name adjustment
    global_tickers = [i.replace("GFREGIOO.MX", "RA.MX") for i in global_tickers]
    global_tickers = [i.replace("MEXCHEM.MX", "ORBIA.MX") for i in global_tickers]
    global_tickers = [i.replace("LIVEPOLC.1.MX", "LIVEPOLC-1.MX") for i in global_tickers]

    # Remove problematic tickers and cash entries
    [global_tickers.remove(i) for i in ["MXN.MX", "USD.MX", "KOFL.MX", "KOFUBL.MX",
                                        "BSMXB.MX", "SITESB.1.MX", "NEMAKA.MX", "NMKA.MX"]]

    return global_tickers

#%%
# Get Prices
def f_get_prices(p_tickers, p_fechas):
    # Initial date, no changes
    f_ini = p_fechas[0]

    # Initial date plus 3 days
    f_fin = str(datetime.datetime.strptime(p_fechas[-1], "%Y-%m-%d") + datetime.timedelta(days=3))[:10]

    # Time counter
    inicio = time.time()

    # Yahoo finance data download
    data = yf.download(p_tickers, start=f_ini, end=f_fin, actions=False, group_by="close",
                       interval="1d", auto_adjust=False, prepost=False, threads=True)

    # Time length of process
    tiempo = "It took", round(time.time() - inicio, 2), "seconds."

    # Morph date column
    data_close = pd.DataFrame({i: data[i]["Close"] for i in p_tickers})

    # We assume NAFRTAC rebalance and Yahoo finance close price times align.

    # Only relevant dates
    ic_fechas = sorted(list(set(data_close.index.astype(str).tolist()) & set(p_fechas)))

    # All prices
    precios = data_close.iloc[[int(np.where(data_close.index == i)[0]) for i in ic_fechas]]

    # Order columns
    precios = precios.reindex(sorted(precios.columns), axis=1)

    return {"precios": precios, "tiempo": tiempo}


In [ ]:
"""
# -- --------------------------------------------------------------------------------------------------- -- #
# -- project: A SHORT DESCRIPTION OF THE PROJECT                                                         -- #
# -- script: main.py : python script with the main functionality                                         -- #
# -- author: YOUR GITHUB USER NAME                                                                       -- #
# -- license: GPL-3.0 License                                                                            -- #
# -- repository: YOUR REPOSITORY URL                                                                     -- #
# -- --------------------------------------------------------------------------------------------------- -- #
"""
# %%
# Import other scripts
import data as dt
import functions as fn
import visualizations as vz

# %%
# ---- Base

# Step 1 - Read all the files
data_files = dt.data_files

# Step 2 - Get all the dates
dates = fn.f_dates(p_files=dt.files)

# Display the first 5 dates in the 2 formats
print(dates["i_dates"][0:4])
print(dates["t_dates"][0:4])
#%%
# Step 3 - Get the tickers for the calculations
global_tickers = fn.f_tickers(p_archivos=dt.files, p_data_archivos=data_files)

# Display global tickers
print(global_tickers[0:4])
#%%
# ---- Historical Prices
global_prices = fn.f_get_prices(p_tickers=global_tickers, p_fechas=dates["i_dates"])
precios = global_prices["precios"]




# PASIVA

## PRE

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader as web

## POST

In [ ]:
kay = pd.read_csv("files/a/NAFTRAC_20180131.csv", skiprows=2, header=0)

In [ ]:
def tickin23(start_date, last_date):
    kay = pd.read_csv("files/a/NAFTRAC_20180131.csv", skiprows=2, header=0)
    cash = (kay.iloc[35]["Peso (%)"] + kay.iloc[34]["Peso (%)"] + kay.iloc[16]["Peso (%)"] + kay.iloc[10][
        "Peso (%)"]) / 100 * 1000000

    kay["Ticker"] = [i.replace("*", "") for i in data["Ticker"]]

    # Weight as decimal
    kay["Ticker"] = kay["Ticker"] + '.MX'
    kay = kay.drop([10, 16, 34, 35, 36])
    kay = kay.sort_values(by=['Ticker'])
    cash = cash + sum(kay["Peso (%)"] / 100 * 1000000 * 0.00125)
    dates = f_dates(p_files=dt.files)
    fechi = dates["i_dates"][start_date:last_date]
    weight = np.array(kay["Peso (%)"])
    kay["Ticker"] = kay["Ticker"].replace("LIVEPOLC.1.MX", "LIVEPOLC-1.MX")
    kay["Ticker"] = kay["Ticker"].replace("SITESB.1.MX", "SITESB-1.MX")
    symbols = np.array(kay["Ticker"])
    startp = datetime.datetime.strptime(fechi[0], "%Y-%m-%d") - datetime.timedelta(days=1)
    price_data = web.get_data_yahoo(symbols,
                                    start=startp,
                                    end=fechi[-1], interval='d')
    price_data = price_data["Adj Close"]
    price_data.index.strftime("%Y-%m-%d")
    price_data = price_data.loc[fechi]
    ret_data = price_data.pct_change()
    weighted_returns = (weight * ret_data)
    port_ret = weighted_returns.sum(axis=1) / 100
    capital = 1000000 * (1 + port_ret) - cash
    cumulative_ret = (port_ret + 1).cumprod() - 1
    df_pasiva = pd.DataFrame()
    df_pasiva["Capital"] = capital
    df_pasiva["Rendimiento"] = port_ret
    df_pasiva["Rendimiendo Acumulado"] = cumulative_ret
    df_pasiva.index.names = ['Timestamp']
    df_pasiva.index = df_pasiva.index.strftime("%d-%m-%Y")
    return df_pasiva

In [ ]:
price_data = web.get_data_yahoo(symbols,
                                    start=startp,
                                    end=fechi[-1], interval='d')
price_data = price_data["Adj Close"]

In [ ]:
files = [f[8:-4] for f in listdir(abspath) if isfile(join(abspath, f))]
files = ["NAFTRAC_" + i.strftime("%Y%m%d") for i in sorted(pd.to_datetime(files))]
for i in files:
    # Read the file
    data = pd.read_csv("files/" + i + ".csv", skiprows=2, header=0)
    # Select only not null columns
    data = data.loc[:, pd.notnull(data.columns)]
    # Clean the ticker name for later use with yfinance
    data["Ticker"] = [i.replace("*","") for i in data["Ticker"]]
    # Weight as decimal
    data["Peso (%)"] = [i/100 for i in data["Peso (%)"]]
    # Save current file in dictionary of data
    data_files[i] = data



In [ ]:
price_data
price_data.index.strftime("%Y-%m-%d")
ret_data = price_data.pct_change()
weighted_returns = (weight * ret_data)


In [ ]:
k = pd.concat(data_files).T.loc['Peso (%)', ]
k = pd.DataFrame(k).droplevel(1)
np.array(k.loc[ 'NAFTRAC_20180131' , : ].T)

In [ ]:
k = pd.concat(data_files).T.loc['Peso (%)', ]
k = pd.DataFrame(k).droplevel(1)
k12 = pd.DataFrame( data = k.index, columns = ["12"])
k12["12"] = [i.replace("NAFTRAC_","") for i in k12["12"] ]
k.rename(index=dict(zip(k.index,k12['12'])))
k.index = pd.to_datetime(k.index).strftime('%Y-%m-%d')


In [ ]:
1450/25

In [ ]:
price_data = web.get_data_yahoo(symbols,
                                start=startp,
                                end=fechi[-1], interval='d')

In [ ]:
price_data.index.strftime("%Y-%m-%d")
price_data = price_data.loc[fechi]
ret_data = price_data.pct_change()

In [ ]:
data12 = data_files.sort_values(by=['Ticker'])
data12["Precio"].T

In [ ]:
ret_data

In [ ]:
dates = f_dates(p_files=dt.files)
fechi = dates["i_dates"]
pd.DataFrame(fechi)

In [ ]:
k.loc[["2021-03-31"]]

In [215]:
def tickin23(start_date, last_date):
    kay = pd.read_csv("files/a/NAFTRAC_20180131.csv", skiprows=2, header=0)
    cash = (kay.iloc[35]["Peso (%)"] + kay.iloc[34]["Peso (%)"] + kay.iloc[16]["Peso (%)"] + kay.iloc[10][
        "Peso (%)"]) / 100 * 1000000

    kay["Ticker"] = [i.replace("*", "") for i in data["Ticker"]]

    # Weight as decimal
    kay["Ticker"] = kay["Ticker"] + '.MX'
    kay = kay.drop([10, 16, 34, 35, 36])
    kay = kay.sort_values(by=['Ticker'])
    cash = cash + sum(kay["Peso (%)"] / 100 * 1000000 * 0.00125)
    dates = f_dates(p_files=dt.files)
    fechi = dates["i_dates"][start_date:last_date]
    weight = np.array(kay["Peso (%)"])
    kay["Ticker"] = kay["Ticker"].replace("LIVEPOLC.1.MX", "LIVEPOLC-1.MX")
    kay["Ticker"] = kay["Ticker"].replace("SITESB.1.MX", "SITESB-1.MX")
    symbols = np.array(kay["Ticker"])
    startp = datetime.datetime.strptime(fechi[0], "%Y-%m-%d") - datetime.timedelta(days=1)
    price_data = web.get_data_yahoo(symbols,
                                    start=startp,
                                    end=fechi[-1], interval='d')
    price_data = price_data["Adj Close"]
    price_data.index.strftime("%Y-%m-%d")
    ret_data = price_data.pct_change()
    return ret_data

In [217]:
for i in data_files:
    


array([ 1.69,  1.36,  0.42, 12.69,  2.44,  0.62,  0.49,  7.44,  0.99,
        2.68,  7.6 ,  2.51,  0.9 ,  0.84,  1.63,  7.54, 13.31,  1.77,
        1.54,  1.74,  1.75,  0.43,  0.32,  0.83,  1.78,  1.26,  1.35,
        0.46,  0.55,  6.21,  0.87,  9.51])

In [216]:
tickin23(0,25)

Symbols,AC.MX,ALFAA.MX,ALSEA.MX,AMXL.MX,ASURB.MX,BBAJIOO.MX,BOLSAA.MX,CEMEXCPO.MX,CUERVO.MX,ELEKTRA.MX,...,LIVEPOLC-1.MX,MEGACPO.MX,ORBIA.MX,PE&OLES.MX,PINFRA.MX,Q.MX,SITESB-1.MX,TLEVISACPO.MX,VESTA.MX,WALMEX.MX
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-31,0.000000,0.010381,0.002965,-0.001717,-0.004480,-0.010317,-0.003636,-0.018389,0.016206,-0.006266,...,-0.002967,0.003063,0.001139,0.013201,0.023663,0.036789,-0.006401,0.003518,0.025600,0.029580
2018-02-01,0.015476,0.015839,0.010020,0.011468,0.008752,0.007119,0.009265,-0.009044,0.001181,0.017609,...,0.006678,-0.001292,0.013270,0.007534,-0.003460,0.000000,-0.007158,-0.000909,-0.000390,-0.010935
2018-02-02,0.011521,-0.024020,0.010083,-0.012472,-0.008128,0.007321,-0.009458,-0.008475,-0.000295,-0.005758,...,-0.018460,-0.009641,-0.000187,-0.050276,-0.004103,0.018925,-0.032444,-0.005198,0.002731,0.011056
2018-02-06,-0.033593,-0.023644,-0.014168,-0.013203,-0.030600,-0.014286,-0.004774,-0.019066,-0.007967,-0.001161,...,-0.015428,-0.006886,-0.029753,-0.031763,-0.019806,-0.017307,-0.024590,-0.027041,-0.025292,-0.043525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-27,-0.012347,-0.047886,-0.012318,-0.023597,-0.026812,-0.012176,-0.016977,-0.010801,-0.017564,-0.003992,...,-0.021905,-0.046413,-0.007378,-0.028293,-0.020936,-0.010255,-0.029028,-0.019974,-0.003526,-0.034307
2020-01-28,0.017464,-0.005897,0.004900,0.027433,0.011772,0.014887,0.005914,0.034040,-0.004396,-0.002290,...,0.001567,0.020333,0.012388,0.002808,0.019008,0.003412,0.008969,0.012724,0.012386,0.019129
2020-01-29,0.005534,-0.007676,0.015514,0.024793,0.005054,0.004101,-0.003057,-0.012422,0.017074,-0.000789,...,-0.002726,0.020553,-0.003405,0.021310,0.005343,0.001133,0.047407,-0.010007,0.013399,0.001073


In [ ]:
    weighted_returns = (weight * ret_data)
    port_ret = weighted_returns.sum(axis=1) / 100
    capital = 1000000 * (1 + port_ret) - cash
    cumulative_ret = (port_ret + 1).cumprod() - 1
    df_pasiva = pd.DataFrame()
    df_pasiva["Capital"] = capital
    df_pasiva["Rendimiento"] = port_ret
    df_pasiva["Rendimiendo Acumulado"] = cumulative_ret
    df_pasiva.index.names = ['Timestamp']
    df_pasiva.index = df_pasiva.index.strftime("%d-%m-%Y")
    return df_pasiva